# CNN

In [1]:
import torch
import torch.nn as nn


In [2]:
conv = nn.Conv2d(1,1,11,stride = 4, padding = 0)

In [3]:
conv

Conv2d(1, 1, kernel_size=(11, 11), stride=(4, 4))

In [4]:
inputs = torch.Tensor(1,1,227,227)
out = conv(inputs)
print(out.shape)

torch.Size([1, 1, 55, 55])


In [6]:
input_ = torch.Tensor(1,1,28,28)
conv1 = nn.Conv2d(1,5,5)
pool = nn.MaxPool2d(2)
out = conv1(input_)
out2 = pool(out)
print(out.size(), out2.size())

torch.Size([1, 5, 24, 24]) torch.Size([1, 5, 12, 12])


# MNIST_CNN

input : 1x28x28

layer1 : conv(in_c = 1, out_c = 32, ker_size = 3, stride/padding = 1) + maxpool(ker_size=2, stride = 2)

layer2 : conv(in_c = 32, out_c = 64, ker_size = 3, stride/padding = 1) + maxpool(ker_size = 2, stride = 2)

view(reshape) -> Fully_connented

Cross entropy Loss (output = 10)

In [7]:
#모듈 import

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [16]:
print(device)

cpu


In [11]:
#MNIST dataset

mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

In [12]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = 100, shuffle = True, drop_last = True)

In [14]:
#CNN 모델 생성

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,3,1,1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(32,64,3,1,1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64,10,bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out
        
        

In [17]:
model = CNN().to(device)

In [18]:
learning_rate= 0.001
training_epochs = 15

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [19]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    
    print('epoch: {} cost = {}'.format(epoch+1, avg_cost))


epoch: 1 cost = 0.22016720473766327
epoch: 2 cost = 0.06089558079838753
epoch: 3 cost = 0.04598744586110115
epoch: 4 cost = 0.0367005281150341
epoch: 5 cost = 0.029995271936058998
epoch: 6 cost = 0.026226848363876343
epoch: 7 cost = 0.020718354731798172
epoch: 8 cost = 0.018531668931245804
epoch: 9 cost = 0.015566564165055752
epoch: 10 cost = 0.013058003038167953
epoch: 11 cost = 0.011760164983570576
epoch: 12 cost = 0.0076063587330281734
epoch: 13 cost = 0.008661064319312572
epoch: 14 cost = 0.007199334446340799
epoch: 15 cost = 0.00625068973749876


In [24]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct = torch.argmax(prediction,1)== Y_test
    accuracy = correct.float().mean()
    print("Accuracy : {:.4f}".format(accuracy))

Accuracy : 0.9858


# Save, Load

In [29]:
torch.save(model.state_dict(),"./model/model.pth")

In [30]:
new_net = CNN().to(device) #모형은 똑같아야한다!!!!!!
new_net.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [31]:
 # 테스트셋을 통해 같은 결과가 나오는 것을 확인할 수 있다.
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = new_net(X_test)
    correct = torch.argmax(prediction,1)== Y_test
    accuracy = correct.float().mean()
    print("Accuracy : {:.4f}".format(accuracy))

/Users/dlwlrma71/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/datasets/mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/dlwlrma71/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/datasets/mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy : 0.9858
